In [1]:
import deepquantum as dq
import numpy as np
import torch
print('version',dq.__version__)

version 3.1.1


构建线路

In [2]:
ini_state1 = [1,1]
test_circ1 = dq.QumodeCircuit(nmode=2, init_state=ini_state1, name="test1", cutoff=3, basis=True)
test_circ1.bs_theta([0, 1],np.pi/4, encode=True)
test_circ1.draw()

构建参数，支持batch输入

In [3]:
nparas = 1
batch = 2
data = (torch.tensor([np.pi/4]*batch*nparas).reshape(batch, nparas))
data[1] +=1
print(data)

tensor([[0.7854],
        [1.7854]])


前向函数，is_prob = None/False/True

In [4]:
re1 = test_circ1(data =data, is_prob = None)
print(re1)

re2 = test_circ1(data =data, is_prob = False)
print(re2)

re3 = test_circ1(data =data, is_prob = True)
print(re3)



tensor([[[ 7.0711e-01+0.0000j,  3.0909e-08+0.7071j],
         [-3.0909e-08+0.7071j,  7.0711e-01+0.0000j]],

        [[-2.1296e-01+0.0000j,  4.2709e-08+0.9771j],
         [-4.2709e-08+0.9771j, -2.1296e-01+0.0000j]]])
{|02>: tensor([[-3.0909e-08+0.7071j],
        [ 1.2863e-08-0.2943j]]), |20>: tensor([[ 3.0909e-08+0.7071j],
        [-1.2863e-08-0.2943j]]), |11>: tensor([[ 0.0000+0.j],
        [-0.9093+0.j]])}
{|02>: tensor([[0.5000],
        [0.0866]]), |20>: tensor([[0.5000],
        [0.0866]]), |11>: tensor([[0.0000],
        [0.8268]])}


In [5]:
re1 = test_circ1(data = data, is_prob = None)
test_circ1.measure()




[{|20>: 489, |02>: 535}, {|20>: 95, |11>: 847, |02>: 82}]

In [6]:
test_circ1.measure(with_prob = True)

[{|02>: (517, tensor([0.5000])), |20>: (507, tensor([0.5000]))},
 {|11>: (831, tensor([0.8268])),
  |02>: (89, tensor([0.0866])),
  |20>: (104, tensor([0.0866]))}]

In [7]:
test_circ1.measure(mcmc = True, with_prob= True, shots= 2000)

chain 5: 100%|█████████████████████████████| 399/399 [00:00<00:00, 10143.39it/s]


[{|20>: (1044, tensor([0.5000])),
  |02>: (955, tensor([0.5000])),
  |11>: (1, tensor([0.]))},
 {|20>: (164, tensor([0.0866])),
  |02>: (160, tensor([0.0866])),
  |11>: (1676, tensor([0.8268]))}]

In [8]:
re2 = test_circ1(data =data, is_prob = False)
test_circ1.measure(with_prob = True)

[{|20>: (518, tensor([0.5000])), |02>: (506, tensor([0.5000]))},
 {|11>: (830, tensor([0.8268])),
  |02>: (92, tensor([0.0866])),
  |20>: (102, tensor([0.0866]))}]

In [10]:
re3 = test_circ1(data =data, is_prob = True)
test_circ1.measure(with_prob = True)

[{|02>: (497, tensor([0.5000])), |20>: (527, tensor([0.5000]))},
 {|11>: (869, tensor([0.8268])),
  |20>: (66, tensor([0.0866])),
  |02>: (89, tensor([0.0866]))}]

## 支持叠加态输入

In [13]:
unitary = np.array([[1, 0, 1, -1, 0, 0],
                    [0, 1, 0 ,0, 0, np.sqrt(2)],
                    [1, 0, 0, 1, 1, 0],
                    [-1, 0, 1, 0, 1, 0],
                    [0, 0, 1, 1, -1, 0],
                    [0, np.sqrt(2), 0, 0, 0, -1]])/np.sqrt(3)
test = torch.zeros([2, 3, 3, 3, 3, 3, 3])*1j
ini_state1 = [(np.sqrt(3)/3, [0, 1, 1, 0, 0, 0]), (np.sqrt(3)/3*1j, [0, 1, 0, 1, 0, 0]), (np.sqrt(3)/3,[0,0,0,0,1,1])]
ini_state2 = [(np.sqrt(2)/2, [0, 1, 1, 0, 0, 0]), (np.sqrt(2)/2*1j, [0, 1, 0, 1, 0, 0])]
test[0] = dq.photonic.state.FockState(state=ini_state1, basis=False).state  # 转为张量
test[1] = dq.photonic.state.FockState(state=ini_state2, basis=False).state
nmode = 6
dq_gate = dq.QumodeCircuit(nmode=nmode, init_state=test, name="test", basis=False)
dq_gate.any(unitary, list(range(nmode)))
dq_gate.draw()

In [18]:
c = dq_gate()

a = dq_gate.measure(with_prob=True)
print(a)

b = sum(value[1] for value in a[1].values())
print(b)

[{|001001>: (118, tensor(0.1111)), |100001>: (150, tensor(0.1481)), |000011>: (284, tensor(0.2899)), |010100>: (208, tensor(0.2159)), |011000>: (135, tensor(0.1111)), |010010>: (41, tensor(0.0434)), |110000>: (83, tensor(0.0741)), |000101>: (5, tensor(0.0064))}, {|010010>: (143, tensor(0.1111)), |000011>: (244, tensor(0.2222)), |000101>: (110, tensor(0.1111)), |001001>: (119, tensor(0.1111)), |110000>: (107, tensor(0.1111)), |100001>: (202, tensor(0.2222)), |010100>: (59, tensor(0.0556)), |011000>: (40, tensor(0.0556))}]
tensor(1.0000)


In [19]:
c = dq_gate(is_prob = True)
print(torch.sum(c))

a = dq_gate.measure(with_prob=True)
print(a)

b = sum(value[1] for value in a[1].values())
print(b)

tensor(2.)
[{|000011>: (295, tensor(0.2899)), |001001>: (107, tensor(0.1111)), |010100>: (217, tensor(0.2159)), |100001>: (160, tensor(0.1481)), |010010>: (58, tensor(0.0434)), |011000>: (118, tensor(0.1111)), |110000>: (60, tensor(0.0741)), |000101>: (9, tensor(0.0064))}, {|110000>: (101, tensor(0.1111)), |010100>: (54, tensor(0.0556)), |000011>: (215, tensor(0.2222)), |000101>: (113, tensor(0.1111)), |100001>: (234, tensor(0.2222)), |001001>: (126, tensor(0.1111)), |010010>: (124, tensor(0.1111)), |011000>: (57, tensor(0.0556))}]
tensor(1.0000)
